# Mike Babb
# babbm@uw.edu
# Find anagrams
## Part 5: Generate a graph of word relationships

In [ ]:
# standard libraries - installed by default
import csv
import os
import sqlite3
import time

In [ ]:
# external libraries - not installed by default
import networkx as nx
import numpy as np
import pandas as pd

In [ ]:
from part_00_process_functions import build_db_conn, query_db

### paths

In [ ]:
# database path and name
db_path = '/project/anagrams/db'
db_name = 'words.db'

In [ ]:
output_file_path = '/project/anagrams/data'
output_file_name = 'sample_graph.gexf'

In [ ]:
ofpn = os.path.join(output_file_path, output_file_name)

### load words

In [ ]:
sql = 'select lcase, word_id from words;'
word_df = query_db(sql=sql, db_path = db_path, db_name = db_name)

In [ ]:
# this is now stored in our the db table

In [ ]:
focal_word_list = ['it', 'tie', 'item', 'mite', 'time', 'emit', 'rite', 'terminator']

In [ ]:
# use dictionary comprehension to create a very fast lookup table
word_id_dict = {}
for word, word_id in zip(word_df['lcase'], word_df['word_id']):
    word_id_dict[word] = word_id 
    word_id_dict[word_id] = word    

In [ ]:
focal_word_id_list = [word_id_dict[focal_word] for focal_word in focal_word_list]

### load 'from' words

In [ ]:
# get parent words
sql = 'select from_word_id, to_word_id from anagrams where to_word_id = (?);'

In [ ]:
# get the parent words
pw_df_list = []
for focal_word_id in focal_word_id_list:
    pw_df = query_db(sql = sql, db_path = db_path, db_name = db_name, params = (focal_word_id,))
    pw_df_list.append(pw_df)

In [ ]:
pw_df = pd.concat(pw_df_list)

In [ ]:
# get the parent words
pw_df['from_word'] = pw_df['from_word_id'].map(word_id_dict)
pw_df['to_word'] = pw_df['to_word_id'].map(word_id_dict)

### load 'to' words

In [ ]:
# get child words
sql = 'select from_word_id, to_word_id from anagrams where from_word_id = (?);'

In [ ]:
cw_df_list = []
for focal_word_id in focal_word_id_list:
    cw_df = query_db(sql = sql, db_path = db_path, db_name = db_name, params = (focal_word_id,))    
    cw_df_list.append(cw_df)

In [ ]:
cw_df = pd.concat(cw_df_list)

In [ ]:
cw_df.head()

In [ ]:
cw_df['from_word'] = cw_df['from_word_id'].map(word_id_dict)
cw_df['to_word'] = cw_df['to_word_id'].map(word_id_dict)

In [ ]:
# concatente
anagram_df = pd.concat([pw_df, cw_df])

In [ ]:
dir(nx)

In [ ]:
anagram_df = anagram_df.drop_duplicates()

In [ ]:
# number of edges:
len(anagram_df)

In [ ]:
# how many nodes?
my_set = set(anagram_df['from_word'].unique().tolist()).union(anagram_df['to_word'].unique().tolist()                                                             )
len(my_set)

### focus only on a few words

In [ ]:
# let's focus on just our words of interest
anagram_df = anagram_df.loc[anagram_df['from_word_id'].isin(focal_word_id_list), ]
anagram_df = anagram_df.loc[anagram_df['to_word_id'].isin(focal_word_id_list), ]

### create and save graph

In [ ]:
# create a directed graph using networkx
# https://networkx.org/
my_graph = nx.from_pandas_edgelist(df=anagram_df,source = 'from_word',
                                    target = 'to_word', create_using = nx.DiGraph())

In [ ]:
# save to disk as a GEXF for visualization in Gephi
# https://gephi.org/
nx.write_gexf(G=my_graph, path = ofpn)